In [1]:
import boto3
import json
import time

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)
elbv2 = boto3.client('elbv2')

### Create VPC - Applicance

In [ ]:
# Create VPC
vpc_applic = ec2.create_vpc(CidrBlock='192.168.1.0/24')
vpc_applic.create_tags(Tags=[{"Key": "Name", "Value": "Applicance"}]);

In [ ]:
# Create 6 subnets in VPC Applicance
sn_appliance_transit_a = ec2.create_subnet(
    VpcId=vpc_applic.id,
    CidrBlock='192.168.1.0/28',
    AvailabilityZone=region_aws+'a'
)
sn_appliance_transit_b = ec2.create_subnet(
    VpcId=vpc_applic.id,
    CidrBlock='192.168.1.48/28',
    AvailabilityZone=region_aws+'b'
)
sn_appliance_server_a = ec2.create_subnet(
    VpcId=vpc_applic.id,
    CidrBlock='192.168.1.16/28',
    AvailabilityZone=region_aws+'a'
)
sn_appliance_server_b = ec2.create_subnet(
    VpcId=vpc_applic.id,
    CidrBlock='192.168.1.64/28',
    AvailabilityZone=region_aws+'b'
)
sn_appliance_nat_a = ec2.create_subnet(
    VpcId=vpc_applic.id,
    CidrBlock='192.168.1.32/28',
    AvailabilityZone=region_aws+'a'
)
sn_appliance_nat_b = ec2.create_subnet(
    VpcId=vpc_applic.id,
    CidrBlock='192.168.1.80/28',
    AvailabilityZone=region_aws+'b'
)

In [ ]:
# Tag subnets
sn_appliance_transit_a.create_tags(Tags=[{"Key": "Name", "Value": "Applicance-Transit-Gateway-a"}])
sn_appliance_transit_b.create_tags(Tags=[{"Key": "Name", "Value": "Applicance-Transit-Gateway-b"}])
sn_appliance_server_a.create_tags(Tags=[{"Key": "Name", "Value": "Applicance-Server-a"}])
sn_appliance_server_b.create_tags(Tags=[{"Key": "Name", "Value": "Applicance-Server-b"}])
sn_appliance_nat_a.create_tags(Tags=[{"Key": "Name", "Value": "Applicance-NAT-a"}])
sn_appliance_nat_b.create_tags(Tags=[{"Key": "Name", "Value": "Applicance-NAT-b"}])

In [ ]:
# Create route table for the VPC Applicance
rt_appliance_transit_a = vpc_applic.create_route_table()
rt_appliance_transit_b = vpc_applic.create_route_table()
rt_appliance_server_a = vpc_applic.create_route_table()
rt_appliance_server_b = vpc_applic.create_route_table()
rt_appliance_nat_a = vpc_applic.create_route_table()
rt_appliance_nat_b = vpc_applic.create_route_table()

# Tags Route Tables
rt_appliance_transit_a.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-2-Transit-Gateway-a"}])
rt_appliance_transit_b.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-2-Transit-Gateway-b"}])
rt_appliance_server_a.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-1-Server-a"}])
rt_appliance_server_b.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-2-Server-b"}])
rt_appliance_nat_a.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-1-NAT-a"}])
rt_appliance_nat_b.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-2-NAT-b"}])

In [ ]:
# Attach the route table to the subnets
rt_appliance_transit_a.associate_with_subnet(SubnetId=sn_appliance_transit_a.id)
rt_appliance_transit_b.associate_with_subnet(SubnetId=sn_appliance_transit_b.id)
rt_appliance_server_a.associate_with_subnet(SubnetId=sn_appliance_server_a.id)
rt_appliance_server_b.associate_with_subnet(SubnetId=sn_appliance_server_b.id)
rt_appliance_nat_a.associate_with_subnet(SubnetId=sn_appliance_nat_a.id)
rt_appliance_nat_b.associate_with_subnet(SubnetId=sn_appliance_nat_b.id)

In [ ]:
# Create Internet Gateway
igw = ec2.create_internet_gateway()

# Attach Internet Gateway to VPC Applicance
vpc_applic.attach_internet_gateway(InternetGatewayId=igw.id);

In [ ]:
# Crear una Elastic IP address
elastic_ip_1 = ec2_client.allocate_address(Domain='vpc')
elastic_ip_2 = ec2_client.allocate_address(Domain='vpc')

In [ ]:
# Crear un NAT gateway en VPC Applicance
natgateway_1 = ec2_client.create_nat_gateway(
    AllocationId=elastic_ip_1['AllocationId'],
    SubnetId=sn_appliance_nat_a.id
)
natgateway_2 = ec2_client.create_nat_gateway(
    AllocationId=elastic_ip_2['AllocationId'],
    SubnetId=sn_appliance_nat_b.id
)

In [ ]:
# Añada una ruta a la tabla de enrutamiento de la VPC Applicance para el internet natgateway
rt_appliance_nat_a.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    GatewayId=igw.id
)
rt_appliance_nat_b.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    GatewayId=igw.id
)

ec2.Route(route_table_id='rtb-0af8aab9aa7278e49', destination_cidr_block='0.0.0.0/0')

In [ ]:
# Añada una ruta a la tabla de enrutamiento de la VPC Applicance para el NAT gateway
rt_appliance_server_a.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    NatGatewayId=natgateway_1['NatGateway']['NatGatewayId']
)
rt_appliance_server_b.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    NatGatewayId=natgateway_2['NatGateway']['NatGatewayId']
)

ec2.Route(route_table_id='rtb-0a25d5ccbce3fd7b7', destination_cidr_block='0.0.0.0/0')

In [ ]:
# Create a security group
sg_appliance = ec2.create_security_group(
    GroupName='ApplicanceVPC-Bastion-SG',
    Description='Access to bastion instance: allow SSH and ICMP access from appropriate location. Allow all traffic from VPC CIDR',
    VpcId=vpc_applic.id
)

# Add rules to the security group
sg_appliance.authorize_ingress(
    CidrIp='0.0.0.0/0',
    IpProtocol='tcp',
    FromPort=22,
    ToPort=22
)
sg_appliance.authorize_ingress(
    CidrIp='0.0.0.0/0',
    IpProtocol='icmp',
    FromPort=-1,
    ToPort=-1
)

In [ ]:
# Create Gateway Load Balancer
glb = elbv2.create_load_balancer(
    Name='ApplicanceVPC-GLB',
    Type='gateway',
    Subnets=[
        sn_appliance_server_a.id, 
        sn_appliance_server_b.id
        ]
)

# Tag Gateway Load Balancer
elbv2.add_tags(
    ResourceArns=[
        glb['LoadBalancers'][0]['LoadBalancerArn']
    ],
    Tags=[
        {
            'Key': 'Name',
            'Value': 'ApplicanceVPC-GLB'
        }
    ]
)

In [ ]:
# Create Target Group
tg = elbv2.create_target_group(
    Name='ApplicanceVPC-TG',
    Port=6081,
    Protocol='GENEVE',
    TargetGroupAttributes=[
        {
            'Key': 'deregistration_delay.timeout_seconds',
            'Value': '20'
        }
    ],
    VpcId=vpc_applic.id,
    HealthCheckPort='traffic-port',
    HealthCheckProtocol='GENEVE'
)

# Tag Target Group
elbv2.add_tags(
    ResourceArns=[
        tg['TargetGroups'][0]['TargetGroupArn']
    ],
    Tags=[
        {
            'Key': 'Name',
            'Value': 'ApplicanceVPC-TG'
        }
    ]
)

# Attach Target Group to Instance
elbv2.register_targets(
    TargetGroupArn=tg['TargetGroups'][0]['TargetGroupArn'],
    Targets=[
        {
            'Id': instance.id
        }
    ]
)



In [ ]:
# Create a Transit Gateway
transit_gateway = ec2_client.create_transit_gateway(
    Description='Transit Gateway',
    Options={
        'AmazonSideAsn': 64512,
        'AutoAcceptSharedAttachments': 'enable',
        'DefaultRouteTableAssociation': 'disable',
        'DefaultRouteTablePropagation': 'enable',
        'DnsSupport': 'enable',
        'VpnEcmpSupport': 'enable'
    }
)

In [ ]:
# Tag Transit Gateway
ec2_client.create_tags(
    Resources=[
        transit_gateway['TransitGateway']['TransitGatewayId'],
    ],
    Tags=[
        {
            'Key': 'Name',
            'Value': 'Transit Gateway'
        },
    ]
);

In [ ]:
# Check the status of the Transit Gateway
transit_gateway_id = transit_gateway['TransitGateway']['TransitGatewayId']
transit_gateway_status = ec2_client.describe_transit_gateways(
    TransitGatewayIds=[transit_gateway_id]
)['TransitGateways'][0]['State']
print("Creating Transit Gateway: {}...".format(transit_gateway_id))
while transit_gateway_status != 'available':
    time.sleep(15)
    transit_gateway_status = ec2_client.describe_transit_gateways(
        TransitGatewayIds=[transit_gateway_id]
    )['TransitGateways'][0]['State']
print("Transit Gateway status: " + transit_gateway_status)

Creating Transit Gateway: tgw-001448bfcd69e57c9...
Transit Gateway status: available


--------------------------------------------------

In [2]:
vpc_spoke1 = ec2.create_vpc(CidrBlock='10.0.0.0/24')
vpc_spoke2 = ec2.create_vpc(CidrBlock='10.0.1.0/24')


# Tag VPCs
vpc_spoke1.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-1"}])
vpc_spoke2.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-2"}])


In [3]:
# Create 4 subnets in VPC 1
sn_spoke1_server_a = ec2.create_subnet(
    VpcId=vpc_spoke1.id,
    CidrBlock='10.0.0.32/28',
    AvailabilityZone=region_aws+'a'
)
sn_spoke1_server_b = ec2.create_subnet(
    VpcId=vpc_spoke1.id,
    CidrBlock='10.0.0.0/28',
    AvailabilityZone=region_aws+'b'
)
sn_spoke1_transit_a = ec2.create_subnet(
    VpcId=vpc_spoke1.id,
    CidrBlock='10.0.0.48/28',
    AvailabilityZone=region_aws+'a'
)
sn_spoke1_transit_b = ec2.create_subnet(
    VpcId=vpc_spoke1.id,
    CidrBlock='10.0.0.16/28',
    AvailabilityZone=region_aws+'b'
)

In [4]:
# Create 4 subnets in VPC 2
sn_spoke2_server_a = ec2.create_subnet(
    VpcId=vpc_spoke2.id, 
    CidrBlock='10.0.1.0/28',
    AvailabilityZone=region_aws+'a'
)
sn_spoke2_server_b = ec2.create_subnet(
    VpcId=vpc_spoke2.id,
    CidrBlock='10.0.1.32/28',
    AvailabilityZone=region_aws+'b'
)
sn_spoke2_transit_a = ec2.create_subnet(
    VpcId=vpc_spoke2.id,
    CidrBlock='10.0.1.48/28',
    AvailabilityZone=region_aws+'a'
)
sn_spoke2_transit_b = ec2.create_subnet(
    VpcId=vpc_spoke2.id,
    CidrBlock='10.0.1.16/28',
    AvailabilityZone=region_aws+'b'
)

In [6]:
# Tags Subnets
sn_spoke1_server_a.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-1-Server-a"}])
sn_spoke1_server_b.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-1-Server-b"}])
sn_spoke1_transit_a.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-1-Transit-Gateway-a"}])
sn_spoke1_transit_b.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-1-Transit-Gateway-b"}])

sn_spoke2_server_a.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-2-Server-a"}])
sn_spoke2_server_b.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-2-Server-b"}])
sn_spoke2_transit_a.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-2-Transit-Gateway-a"}])
sn_spoke2_transit_b.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-2-Transit-Gateway-b"}])



[ec2.Tag(resource_id='subnet-08595cc3f59a6aef7', key='Name', value='Applicance-NAT-b')]

In [7]:
# Create route table for the VPC1
rt_spoke1_server_a = vpc_spoke1.create_route_table()
rt_spoke1_server_b = vpc_spoke1.create_route_table()
rt_spoke1_transit_a = vpc_spoke1.create_route_table()
rt_spoke1_transit_b = vpc_spoke1.create_route_table()

# Tags Route Tables
rt_spoke1_server_a.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-1-Server-a"}])
rt_spoke1_server_b.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-1-Server-b"}])
rt_spoke1_transit_a.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-1-Transit-Gateway-a"}])
rt_spoke1_transit_b.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-1-Transit-Gateway-b"}])

[ec2.Tag(resource_id='rtb-0d11768faf25cba5a', key='Name', value='Spoke-1-Transit-Gateway-b')]

In [8]:
# Create route table for the vpc2
rt_spoke2_server_a = vpc_spoke2.create_route_table()
rt_spoke2_server_b = vpc_spoke2.create_route_table()
rt_spoke2_transit_a = vpc_spoke2.create_route_table()
rt_spoke2_transit_b = vpc_spoke2.create_route_table()

# Tags Route Tables
rt_spoke2_server_a.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-2-Server-a"}])
rt_spoke2_server_b.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-2-Server-b"}])
rt_spoke2_transit_a.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-1-Transit-Gateway-a"}])
rt_spoke2_transit_b.create_tags(Tags=[{"Key": "Name", "Value": "Spoke-2-Transit-Gateway-b"}])

[ec2.Tag(resource_id='rtb-07e10df00443919e8', key='Name', value='Spoke-2-Transit-Gateway-b')]

[ec2.Tag(resource_id='rtb-0af8aab9aa7278e49', key='Name', value='Spoke-2-NAT-b')]

In [11]:
# Attach the route table to the subnets
rt_spoke1_server_a.associate_with_subnet(SubnetId=sn_spoke1_server_a.id)
rt_spoke1_server_b.associate_with_subnet(SubnetId=sn_spoke1_server_b.id)
rt_spoke1_transit_a.associate_with_subnet(SubnetId=sn_spoke1_transit_a.id)
rt_spoke1_transit_b.associate_with_subnet(SubnetId=sn_spoke1_transit_b.id)

rt_spoke2_server_a.associate_with_subnet(SubnetId=sn_spoke2_server_a.id)
rt_spoke2_server_b.associate_with_subnet(SubnetId=sn_spoke2_server_b.id)
rt_spoke2_transit_a.associate_with_subnet(SubnetId=sn_spoke2_transit_a.id)
rt_spoke2_transit_b.associate_with_subnet(SubnetId=sn_spoke2_transit_b.id)



ec2.RouteTableAssociation(id='rtbassoc-027e639f43d1b3633')

In [22]:
# Attach VPC 1 to Transit Gateway
transit_gateway_attachment_spoke1 = ec2_client.create_transit_gateway_vpc_attachment(
    TransitGatewayId=transit_gateway['TransitGateway']['TransitGatewayId'],
    VpcId=vpc_spoke1.id,
    SubnetIds=[
        sn_spoke1_transit_a.id,
        sn_spoke1_transit_b.id
    ],
    Options={
        'ApplianceModeSupport': 'enable',
        'DnsSupport': 'enable',
        'Ipv6Support': 'disable'
    }
)

In [23]:
# Tag Transit Gateway Attachment VPC 1
ec2_client.create_tags(
    Resources=[
        transit_gateway_attachment_spoke1['TransitGatewayVpcAttachment']['TransitGatewayAttachmentId'],
    ],
    Tags=[
        {
            'Key': 'Name',
            'Value': 'Spoke 1 VPC Association'
        },
    ]
);

In [24]:
# Attach VPC 2 to Transit Gateway
transit_gateway_attachment_spoke2 = ec2_client.create_transit_gateway_vpc_attachment(
    TransitGatewayId=transit_gateway['TransitGateway']['TransitGatewayId'],
    VpcId=vpc_spoke2.id,
    SubnetIds=[
        sn_spoke2_transit_a.id,
        sn_spoke2_transit_b.id
    ],
    Options={
        'ApplianceModeSupport': 'enable',
        'DnsSupport': 'enable',
        'Ipv6Support': 'disable'
    }
)

In [25]:
# Tag Transit Gateway Attachment VPC 2
ec2_client.create_tags(
    Resources=[
        transit_gateway_attachment_spoke2['TransitGatewayVpcAttachment']['TransitGatewayAttachmentId'],
    ],
    Tags=[
        {
            'Key': 'Name',
            'Value': 'Spoke 2 VPC Association'
        },
    ]
);

In [26]:
# Attach VPC 3 to Transit Gateway
transit_gateway_attachment_applic = ec2_client.create_transit_gateway_vpc_attachment(
    TransitGatewayId=transit_gateway['TransitGateway']['TransitGatewayId'],
    VpcId=vpc_applic.id,
    SubnetIds=[
        sn_appliance_transit_a.id,
        sn_appliance_transit_b.id
    ],
    Options={
        'ApplianceModeSupport': 'enable',
        'DnsSupport': 'enable',
        'Ipv6Support': 'disable'
    }
)

In [27]:
# Tag Transit Gateway Attachment VPC 3
ec2_client.create_tags(
    Resources=[
        transit_gateway_attachment_applic['TransitGatewayVpcAttachment']['TransitGatewayAttachmentId'],
    ],
    Tags=[
        {
            'Key': 'Name',
            'Value': 'Applicance VPC Association'
        },
    ]
);

In [28]:
# Create Transit Gateway Route Table: Egress
egress_tg_rt = ec2_client.create_transit_gateway_route_table(
    TransitGatewayId=transit_gateway['TransitGateway']['TransitGatewayId']
)
# Tag the Transit Gateway Route Table
ec2_client.create_tags(
    Resources=[
        egress_tg_rt['TransitGatewayRouteTable']['TransitGatewayRouteTableId']
    ],
    Tags=[
        {
            'Key': 'Name',
            'Value': 'Egress Route Table'
        },
    ]
);

In [29]:
# Create Transit Gateway Route Table: Transit
transit_tg_rt = ec2_client.create_transit_gateway_route_table(
    TransitGatewayId=transit_gateway['TransitGateway']['TransitGatewayId']
)
# Tag the Transit Gateway Route Table
ec2_client.create_tags(
    Resources=[
        transit_tg_rt['TransitGatewayRouteTable']['TransitGatewayRouteTableId']
    ],
    Tags=[
        {
            'Key': 'Name',
            'Value': 'Transit Route Table'
        },
    ]
);

In [32]:
print("Creating Transit Gateway Route Tables...")
time.sleep(30)

Creating Transit Gateway Route Tables...


In [33]:
# Associate Route Table: Egress with VPC 3
ec2_client.associate_transit_gateway_route_table(
    TransitGatewayRouteTableId=egress_tg_rt['TransitGatewayRouteTable']['TransitGatewayRouteTableId'],
    TransitGatewayAttachmentId=transit_gateway_attachment_applic['TransitGatewayVpcAttachment']['TransitGatewayAttachmentId']
);

In [34]:
# Associate Route Table: Transit with VPC 1
ec2_client.associate_transit_gateway_route_table(
    TransitGatewayRouteTableId=transit_tg_rt['TransitGatewayRouteTable']['TransitGatewayRouteTableId'],
    TransitGatewayAttachmentId=transit_gateway_attachment_spoke1['TransitGatewayVpcAttachment']['TransitGatewayAttachmentId']
);

In [35]:
# Associate Route Table: Transit with VPC 2
ec2_client.associate_transit_gateway_route_table(
    TransitGatewayRouteTableId=transit_tg_rt['TransitGatewayRouteTable']['TransitGatewayRouteTableId'],
    TransitGatewayAttachmentId=transit_gateway_attachment_spoke2['TransitGatewayVpcAttachment']['TransitGatewayAttachmentId']
);

In [37]:
rt_spoke1_server_a.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    TransitGatewayId=transit_gateway['TransitGateway']['TransitGatewayId']
)
rt_spoke1_server_b.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    TransitGatewayId=transit_gateway['TransitGateway']['TransitGatewayId']
)
rt_spoke1_transit_a.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    TransitGatewayId=transit_gateway['TransitGateway']['TransitGatewayId']
)
rt_spoke1_transit_b.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    TransitGatewayId=transit_gateway['TransitGateway']['TransitGatewayId']
)

ec2.Route(route_table_id='rtb-0d11768faf25cba5a', destination_cidr_block='0.0.0.0/0')

In [38]:
rt_spoke2_server_a.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    TransitGatewayId=transit_gateway['TransitGateway']['TransitGatewayId']
)
rt_spoke2_server_b.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    TransitGatewayId=transit_gateway['TransitGateway']['TransitGatewayId']
)
rt_spoke2_transit_a.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    TransitGatewayId=transit_gateway['TransitGateway']['TransitGatewayId']
)
rt_spoke2_transit_b.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    TransitGatewayId=transit_gateway['TransitGateway']['TransitGatewayId']
)

ec2.Route(route_table_id='rtb-07e10df00443919e8', destination_cidr_block='0.0.0.0/0')

In [39]:
rt_appliance_transit_a.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    TransitGatewayId=transit_gateway['TransitGateway']['TransitGatewayId']
)
rt_appliance_transit_b.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    TransitGatewayId=transit_gateway['TransitGateway']['TransitGatewayId']
)

ec2.Route(route_table_id='rtb-0f09ce72fd810352b', destination_cidr_block='0.0.0.0/0')

In [43]:
print("En la terminal ejecute el siguiente comando:")
print("    python create_ec2_ssm.py --vpc {} --subnet {} --tag SSM-Instance-Spoke1".format(vpc_spoke1.id, sn_spoke1_server_a.id))

En la terminal ejecute el siguiente comando:
    python create_ec2_ssm.py --vpc vpc-0c2b46ce58febf19f --subnet subnet-01fdedac7f46223ef --tag SSM-Instance-Spoke1


'vpc-0c2b46ce58febf19f'